In [1]:
import pandas as pd 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import faiss


/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-19 12:10:44.358574: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-19 12:10:44.711512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750324244.826430   82428 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750324244.863379   82428 cuda_blas.

In [2]:
df = pd.read_csv("movies_list.csv")
df

,year,genre,director,time,page_url,movie_title,description,image_url,time_minutes
0,1994,драма,Фрэнк Дарабонт,1 час 48 минут,https://www.film.ru/movies/pobeg-iz-shoushenka,Побег из Шоушенка,Банкира Энди Дюфрейна обвиняют в убийстве жены...,https://www.film.ru/sites/default/files/styles...,108
1,1994,драма,Квентин Тарантино,2 часа 19 минут,https://www.film.ru/movies/kriminalnoe-chtivo,Криминальное чтиво,Знаковый фильм Квентина Тарантино в новом рема...,https://www.film.ru/sites/default/files/styles...,139
2,1991,драма,Джонатан Демме,1 час 58 минут,https://www.film.ru/movies/molchanie-yagnyat,Молчание ягнят,Психопат похищает и убивает молодых женщин по ...,https://www.film.ru/sites/default/files/styles...,118
3,2001,мультфильмы,Хаяо Миядзаки,2 часа 5 минут,https://www.film.ru/movies/unesennye-prizrakami,Унесенные призраками,Маленькая Тихиро вместе с мамой и папой переез...,https://www.film.ru/sites/default/files/styles...,125
4,1985,комедия,Роберт Земекис,1 час 48 минут,https://www.film.ru/movies/nazad-v-budushchee,Назад в будущее,"Марти МакФлай, типичный американский тинейджер...",https://www.film.ru/sites/default/files/styles...,108
...,...,...,...,...,...,...,...,...,...
15751,1941,драма,Эдмунд Гулдинг,1 час 48 минут,https://www.film.ru/movies/velikaya-lozh-0,Великая ложь,"Проснувшись после пьяной свадьбы Питузнает, чт...",https://www.film.ru/sites/default/files/styles...,108
15752,1940,комедия,Джордж Маршалл,1 час 25 минут,https://www.film.ru/movies/ohotniki-za-privide...,Охотники за привидениями,"Мэри Картер унаследовала наКубе замок, населен...",https://www.film.ru/sites/default/files/styles...,85
15753,1949,драма,Кинг Видор,1 час 54 минуты,https://www.film.ru/movies/istochnik-3,Источник,Архитектор Говард Роарк—творец «новой волны» в...,https://www.film.ru/sites/default/files/styles...,114
15754,1942,биографический,Лесли Говард,1 час 58 минут,https://www.film.ru/movies/istoriya-istrebitel...,История истребителя Спитфайер,История создания легендарного истребителя Спит...,https://www.film.ru/sites/default/files/styles...,118


In [3]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2") 

/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
embeddings = model.encode(df['description'].tolist(),show_progress_bar=True)

Batches: 100%|██████████| 493/493 [00:27<00:00, 18.17it/s]


In [5]:
embeddings = np.array(embeddings).astype("float32")

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [8]:
# Загрузка данных
df = pd.read_csv("movies_list.csv")

# Предобработка описаний
df["description"] = df["description"].fillna("")

# Загрузка модели
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Генерация эмбеддингов
embeddings = model.encode(df["description"].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# Нормализация для косинусного сходства
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

# Создание индекса на основе скалярного произведения (эквивалент косинусному сходству при нормализации)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product == Cosine similarity при нормализации
index.add(embeddings)

# Сохранение
np.save("movie_vectors.npy", embeddings)
faiss.write_index(index, "index.bin")

/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 493/493 [00:25<00:00, 19.21it/s]


In [18]:
# Запрос пользователя
user_query = "Новогодняя ночь"
query_vector = model.encode([user_query],convert_to_numpy=True).astype("float32")

# Векторизуем запрос
#query_embedding = model.encode([user_query])

# Считаем схожесть запроса с каждым сериалом
#similarities = cosine_similarity(query_embedding, embeddings)[0]  # получаем 1D-массив

# Получим индексы топ-5 самых похожих сериалов
#top_n = 5
#top_indices = similarities.argsort()[-top_n:][::-1]  # сортировка по убыванию

# Выводим названия и описания похожих сериалов
# for i in top_indices:
#     print(f"\n🎬 Название: {df.iloc[i]['tvshow_title']}")
#     print(f"📄 Описание: {df.iloc[i]['description']}")
#     print(f"🔍 Схожесть: {similarities[i]:.3f}")

k = 9
distances, indices = index.search(query_vector, k)

# Выводим результаты
for i in indices[0]:
    print(f"\n🎬 Название: {df.iloc[i]['movie_title']}")
    print(f"📄 Описание: {df.iloc[i]['description']}")


🎬 Название: С Новым годом!
📄 Описание: Друзья собираются, чтобы отпраздновать Новый год. В ходе вечера они решают сыграть в игру и положить свои телефоны на общее обозрение. Во время игры раскроется много секретов, разобьются сердца и сломаются судьбы. С Новым годом!

🎬 Название: О чем еще говорят мужчины
📄 Описание: Так случилось, что почти весь день 31 декабря наши герои проводят вместе, Как они проводят время накануне Нового года? Разумеется, в разговорах. О чем? Во-первых, конечно, о женщинах – тема-то неисчерпаемая. А еще о…

🎬 Название: Обратная связь
📄 Описание: Канун Нового года. Старая компания снова в сборе и собирается весело отметить праздник. Но атмосфера в доме вновь накаляется до предела.

🎬 Название: Декалог, три
📄 Описание: Канун Рождества - ночь, когда вся семья вместе, и никто не хочет оставаться один. Эва соблазняет Януша, ее бывшего любовника, под любым предлогом выйти из дома и пробыть с ней всю ночь. Януш хочет вернуться домой, но Эва определенно решительна… 

🎬